In [3]:
import urllib
from sqlalchemy import create_engine
from langchain.sql_database import SQLDatabase

# def get_connection_string():
    
#     conn_string = ('Driver={ODBC Driver 17 for SQL Server};'
#                   'Server=tcp:mysqlserver16666.database.windows.net,1433;'
#                   'Database=GMBOT;'
#                   'Uid=Azureuser;'
#                   'Pwd={Azure@23498};')
#     return conn_string


server = 'mysqlserver16666.database.windows.net'
database = 'GMBOT'
username = 'Azureuser'
password = 'Azure@23498'   
driver= '{ODBC Driver 17 for SQL Server}'

conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""

params = urllib.parse.quote_plus(conn)
conn_str = 'mssql+pyodbc:///?autocommit=true&odbc_connect={}'.format(params)

db = SQLDatabase.from_uri(conn_str)

In [4]:
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain.agents.agent_types import AgentType

In [5]:
import os

os.environ['OPENAI_API_VERSION'] = '2023-12-01-preview'
os.environ['AZURE_OPENAI_API_KEY'] = 'e63ed695495543d58595fab4e27e4ff1'

In [6]:
llm = AzureChatOpenAI(azure_deployment='gpt-16k',azure_endpoint='https://tv-llm-applications.openai.azure.com/')
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025621B0BC40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025621B29150>, openai_api_key='e63ed695495543d58595fab4e27e4ff1', openai_proxy='', azure_endpoint='https://tv-llm-applications.openai.azure.com/', deployment_name='gpt-16k', openai_api_version='2023-12-01-preview', openai_api_type='azure')

In [7]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS
)

In [8]:
question = "How many records are present in pay slips table?"

In [9]:
agent_executor.invoke(question)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


EmployeeInformation, EmployeeLeave, EmployeeSchedule, payslips, sysdiagrams
Invoking: `sql_db_schema` with `payslips`



CREATE TABLE payslips (
	[EmployeeName] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[EmployeeID] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Salary] DECIMAL(10, 2) NOT NULL, 
	[Deductions] DECIMAL(10, 2) NOT NULL, 
	[NetPay] DECIMAL(10, 2) NOT NULL
)

/*
3 rows from payslips table:
EmployeeName	EmployeeID	Salary	Deductions	NetPay
John Doe	JO1234	5000.00	200.00	4800.00
Chris Adam	CA1234	6000.00	250.00	5750.00
Peter Jones	PJ1234	5500.00	220.00	5280.00
*/There are 3 records present in the payslips table.

> Finished chain.


{'input': 'How many records are present in pay slips table?',
 'output': 'There are 3 records present in the payslips table.'}

In [1]:
# Import Statements
import urllib
from langchain.sql_database import SQLDatabase

from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain.agents.agent_types import AgentType

import os

os.environ['OPENAI_API_VERSION'] = '2023-12-01-preview'
os.environ['AZURE_OPENAI_API_KEY'] = 'e63ed695495543d58595fab4e27e4ff1'

def get_connection_string():
    server = 'mysqlserver16666.database.windows.net'
    database = 'GMBOT'
    username = 'Azureuser'
    password = 'Azure@23498'   
    driver= '{ODBC Driver 17 for SQL Server}'

    conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
    Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""

    params = urllib.parse.quote_plus(conn)
    conn_str = 'mssql+pyodbc:///?autocommit=true&odbc_connect={}'.format(params)

    return conn_str

def sql_query(question):
    conn_str = get_connection_string()

    db = SQLDatabase.from_uri(conn_str)

    sql_llm = AzureChatOpenAI(azure_deployment='gpt-16k',
                              azure_endpoint='https://tv-llm-applications.openai.azure.com/')

    toolkit = SQLDatabaseToolkit(db=db, llm=sql_llm)
    agent_executor = create_sql_agent(
        llm=sql_llm,
        toolkit=toolkit,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS
    )

    response = agent_executor.invoke(question)

    return response

In [2]:
sql_query('How many employees are present in the organization?')



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


EmployeeInformation, EmployeeLeave, EmployeeSchedule, payslips, sysdiagrams
Invoking: `sql_db_schema` with `EmployeeInformation`



CREATE TABLE [EmployeeInformation] (
	[EmployeeID] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[EmployeeName] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Age] INTEGER NULL, 
	[Gender] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Contact] VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Email] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Address] VARCHAR(100) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Profession] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[EmploymentType] VARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK__Employee__7AD04FF1784B8A46] PRIMARY KEY ([EmployeeID])
)

/*
3 rows from EmployeeInformation table:
EmployeeID	EmployeeName	Age	Gender	Contact	Email	Add

{'input': 'How many employees are present in the organization?',
 'output': 'There are 4 employees present in the organization.'}

In [2]:
import urllib
from langchain.sql_database import SQLDatabase

from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain.agents.agent_types import AgentType

import os

os.environ['OPENAI_API_VERSION'] = '2023-12-01-preview'
os.environ['AZURE_OPENAI_API_KEY'] = 'e63ed695495543d58595fab4e27e4ff1'
def get_connection_string():
    server = 'mysqlserver16666.database.windows.net'
    database = 'GMBOT'
    username = 'Azureuser'
    password = 'Azure@23498'   
    driver= '{ODBC Driver 17 for SQL Server}'

    conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
    Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""

    params = urllib.parse.quote_plus(conn)
    conn_str = 'mssql+pyodbc:///?autocommit=true&odbc_connect={}'.format(params)

    return conn_str

conn_str = get_connection_string()

db = SQLDatabase.from_uri(conn_str)